In [1]:
# import PyPDF2
import os
import numpy as np
import openai
from dotenv import load_dotenv
# from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
# from langchain.embeddings import OpenAIEmbeddings 
from langchain_openai import AzureOpenAIEmbeddings
# from langchain_openai import AzureOpenAIEmbeddings
# from langchain.vectorstores import AzureSearch
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from azure.identity import DefaultAzureCredential
# from azure.search.documents.indexes.models import HnswVectorSearchAlgorithmConfiguration, HnswParameters, VectorSearchAlgorithmConfiguration
import warnings
warnings.filterwarnings('ignore')

from langchain_community.vectorstores.azuresearch import (
    AzureSearch,
    AzureSearchVectorStoreRetriever,
)

In [2]:
llm = AzureChatOpenAI(
    openai_api_key = "7439c61b0745446dbbb029521f498265",
    azure_endpoint = "https://ai-dev-openai-fts.openai.azure.com/",
    openai_api_version = "2023-07-01-preview",
    openai_organization = "azure",
    temperature=0,    
    deployment_name="gpt35")

In [3]:
embeddings = AzureOpenAIEmbeddings(
            openai_api_key = "7439c61b0745446dbbb029521f498265",
            azure_endpoint = "https://ai-dev-openai-fts.openai.azure.com/",
            openai_api_type = "azure",
            openai_api_version = "2023-07-01-preview",
            deployment="text-embedding-ada-002", 
            chunk_size=1
                                )

In [4]:
query = "What did the president say about Justice Breyer"

llm.predict(query)

"As an AI language model, I do not have access to the latest news and updates. However, as of my last training data, I do not have any information about the president's recent comments on Justice Breyer."

In [5]:
import requests
from langchain.document_loaders import TextLoader

url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)
    

loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

documents

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citize

In [6]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [7]:
acs = AzureSearch(azure_search_endpoint="https://ai-dev-ss-fts.search.windows.net",
                 azure_search_key="ICyDmM0S3sWaLpYwNGPfUOz7jblRyOHnE6p7uDHPRRAzSeAaLBFM",
                 index_name="reg-data-ankur",
                 embedding_function= embeddings.embed_query
                 )

In [8]:
# loader = DirectoryLoader(r'C:\Users\achauha2\Downloads\test-txt', glob="*.txt", loader_cls=TextLoader, loader_kwargs={'autodetect_encoding': True})

# documents = loader.load()
# text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=400)
# docs = text_splitter.split_documents(documents)

# # Add documents to Azure Search
# acs.add_documents(documents=docs)

acs.add_documents(documents=chunks)

['YzRjOGVkZmUtY2YzNi00YmQ5LWI3YTAtMzY0MjlmYjE0MjYy',
 'MWFkODdjZDQtNDZhMy00NWIyLWE1OWMtMTM5ZmRhZmJkOTAz',
 'NDdiMTkxMDAtN2RmYi00OWZhLTg5MWItNDkwZTgzMzI4M2Q5',
 'OGM1OTEwMGItNGFiNC00ZDU4LTgwODItNDMyN2ZiYzMwMThh',
 'ZmUwMGVmNzgtMjAwYy00MjUxLWFmZDctMDg5NjI1NGYzODM4',
 'MjM3YTIwYzMtMWI4Ni00MzU0LWJjYjctMDE1ZmJhYzE3Mjli',
 'NzhjZjE5YWMtMzlkZC00Y2I4LWI1NmMtMWVlOThmZTdkZWJm',
 'ZThhYjI3OGEtZWY2NC00OWQ0LTkyNjktZTU3Zjc1OTNiNDkx',
 'MjM1NjI5M2ItZDMxMy00ODYyLWI0NzktYzUwYjg3NDNiYmNl',
 'YjQ0MTU0ZDgtY2FiNi00OTMzLWFmMDgtOTY3ZGU5OTg4YWRh',
 'ODA3MmFhNmEtMTZjZi00NzNhLWI2MzMtMTY1Mzc5MGYxYzY2',
 'ODRmZDU3OGMtOTk3ZS00YjhjLWFkNTEtMzkxZjNkODk0ZjBk',
 'YWYyYjEyN2MtMmMzNC00NTQxLWFjMDEtYmFhMWQzMzJjYTRl',
 'NmQ1N2ZkNjEtZWNiZC00Yjg0LThlZDktY2NlY2MwYjI4ODcz',
 'YWI0OTA1ZjgtNmE0NS00ODYwLWE1NjktNjc2YmQ2MWZhOGMy',
 'NDY0ZWFjMzUtZmQwZS00ZTg1LWEwODctNzlhNjdkOGRkMThm',
 'MmMyOTJkYWItZTRkNC00MWY1LWE2MjUtOGRkYjZhY2ZmMmE0',
 'MDU1ZGIxYzktNjdkZC00MGFjLWJjMjYtZmM1ZTNiMzJiZmI3',
 'MmJmMDYyOTctZWRiNi00MWMxLWFjYmEtMGYyZWViZmMz

In [9]:
retriever = acs.as_retriever(search_kwargs={"k": 3})

In [10]:
from langchain import PromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

prompt = PromptTemplate(
    template=template, 
    input_variables=["context","question"]
  )

print(prompt)

input_variables=['context', 'question'] template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"


In [11]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [12]:
# Ragas wants ['question', 'answer', 'contexts', 'ground_truths'] as
'''
{
    "question": [], <-- question from faq doc
    "answer": [], <-- answer from generated result
    "contexts": [], <-- context
    "ground_truths": [] <-- actual answer
}
'''

from datasets import Dataset

questions = ["What did the president say about Justice Breyer?", 
             "What did the president say about Intel's CEO?",
             "What did the president say about gun violence?",
            ]
ground_truths = [["The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service."],
                ["The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion."],
                ["The president asked Congress to pass proven measures to reduce gun violence."]]
answers = []
contexts = []

# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [13]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["azure_endpoint"] = "https://ai-dev-openai-fts.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "7439c61b0745446dbbb029521f498265"

In [14]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

result

evaluating with [context_precision]


  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: ('Fatal error occurred while running async tasks.', AuthenticationError("Error code: 401 - {'error': {'message': 'Incorrect API key provided: 7439c61b********************8265. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}"))